In [136]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")

from sklearn.preprocessing import OrdinalEncoder

from datgan import stats_assessment
from datgan import ml_assessment, transform_results


# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [176]:
tmp = pd.read_csv('../../data/LPMC/trips_bias.csv')

In [177]:
len(tmp)

8437

In [137]:
df_orig = pd.read_csv('../../data/LPMC/trips.csv', index_col=False)

continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt','dur_driving', 'driving_traffic_percent']

#to_check = list(set(df_orig.columns) - set(['age', 'female', 'hh_region']))
to_check = ['distance', 'travel_mode', 'purpose', 'driving_license', 'faretype', 'hh_vehicles', 'hh_income', 'hh_people']

cond_inputs = list(set(df_orig.columns) - set(to_check))

synth_files = ['../../data/LPMC/trips_bias.csv']
labels = ['bias']

for i in range(5):
    for j in range(5):
        synth_files.append('../../data/synthetic/bias/ciDATGAN_{}_{}.csv'.format(i+1, j+1))
        labels.append('ciDATGAN_{}_{}'.format(i+1, j+1))
        synth_files.append('../../data/synthetic/bias/DATGAN_{}_{}.csv'.format(i+1, j+1))
        labels.append('DATGAN_{}_{}'.format(i+1, j+1))

        synth_files.append('../../data/synthetic/normal/DATGAN_{}_{}.csv'.format(i+1, j+1))
        labels.append('normal_DATGAN_{}_{}'.format(i+1, j+1))

len_df = len(df_orig)

In [138]:
results_path = './results/'

if not os.path.exists(results_path):
    os.makedirs(results_path)

In [139]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
colors = [(0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0), (1.0, 0.0, 0.0, 0.0), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0), (0.6627450980392157, 0.6627450980392157, 0.6627450980392157, 1.0)]

# First level

In [140]:
pickle_name = 'res_lvl1_bias.pickle'
aggregation_level = 1

first_lvl_stats = {}

try:
    first_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickle file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickle file, using that


In [141]:
for i, (f, l) in enumerate(zip(synth_files, labels)):

    file_name = f.split('/')[-1].split('.')[0]

    if l in first_lvl_stats:
        print("Results for \033[1m{}\033[0m ({}/{}) already exists!".format(l, i+1, len(synth_files)))
    else:
        print("Preparing stats for \033[1m{}\033[0m ({}/{})".format(l, i+1, len(synth_files)))

        first_lvl_stats[l] = {}

        df_synth = pd.read_csv(f, index_col=False)

        repl = False
        if len(df_synth) < len_df:
            repl = True

        df_synth = df_synth.sample(len_df, replace=repl).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        first_lvl_stats[l] = stats

    pickle.dump(first_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for bias (1/76) already exists!
Results for ciDATGAN_1_1 (2/76) already exists!
Results for DATGAN_1_1 (3/76) already exists!
Results for normal_DATGAN_1_1 (4/76) already exists!
Results for ciDATGAN_1_2 (5/76) already exists!
Results for DATGAN_1_2 (6/76) already exists!
Results for normal_DATGAN_1_2 (7/76) already exists!
Results for ciDATGAN_1_3 (8/76) already exists!
Results for DATGAN_1_3 (9/76) already exists!
Results for normal_DATGAN_1_3 (10/76) already exists!
Results for ciDATGAN_1_4 (11/76) already exists!
Results for DATGAN_1_4 (12/76) already exists!
Results for normal_DATGAN_1_4 (13/76) already exists!
Results for ciDATGAN_1_5 (14/76) already exists!
Results for DATGAN_1_5 (15/76) already exists!
Results for normal_DATGAN_1_5 (16/76) already exists!
Results for ciDATGAN_2_1 (17/76) already exists!
Results for DATGAN_2_1 (18/76) already exists!
Results for normal_DATGAN_2_1 (19/76) already exists!
Results for ciDATGAN_2_2 (20/76) already exists!
Results for DATGAN_

In [142]:
res = {}

for test in ['all', 'cont', 'cat']:

    res[test] = {}

    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_columns
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_columns)

    cols = set(cols) - set(cond_inputs)

    for s in stats_str:
        res[test][s] = {}

    for m in first_lvl_stats.keys():
        for s in stats_str:
            res[test][s][m] = []

            for c in cols:
                res[test][s][m].append(first_lvl_stats[m][c][s])

In [143]:
avg = {}

for test in ['all', 'cont', 'cat']:

    avg[test] = {}

    for s in stats_str:
        avg[test][s] = {}

        for m in first_lvl_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [153]:
sts = 'srmse'

res_DATGAN = []
res_ciDATGAN = []
res_normal = []

for i in range(5):
    for j in range(5):
        res_DATGAN.append(avg['all'][sts]['DATGAN_{}_{}'.format(i+1, j+1)]['mean'])
        res_ciDATGAN.append(avg['all'][sts]['ciDATGAN_{}_{}'.format(i+1, j+1)]['mean'])
        res_normal.append(avg['all'][sts]['normal_DATGAN_{}_{}'.format(i+1, j+1)]['mean'])

tmp = [res_DATGAN, res_ciDATGAN, res_normal]
df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN', 'normal'])

plt.figure(figsize=(10,7))

sns.boxplot(data=df.T, palette=colors,showmeans=True,
            meanprops={"marker":"o",
                       "markerfacecolor":"white",
                       "markeredgecolor":"black",
                      "markersize":"10"})

val = avg['all'][sts]['bias']['mean']

plt.plot([-0.3, 2.3], [val, val], 'k--')

plt.xticks([0,1,2], ['DATGAN', 'ciDATGAN', 'DATGAN - unbiased'])
plt.ylabel(sts.upper())

plt.savefig('../../figures/bias/lvl1.png', bbox_inches='tight')
plt.savefig('../../figures/bias/lvl1.pdf', bbox_inches='tight')
plt.close()

# Second level

In [154]:
pickle_name = 'res_lvl2_bias.pickle'
aggregation_level = 2

second_lvl_stats = {}

try:
    second_lvl_stats = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

Found previous pickel file, using that


In [155]:
for i, (f, l) in enumerate(zip(synth_files, labels)):

    file_name = f.split('/')[-1].split('.')[0]

    if l in second_lvl_stats:
        print("Results for \033[1m{}\033[0m ({}/{}) already exists!".format(l, i+1, len(synth_files)))
    else:
        print("Preparing stats for \033[1m{}\033[0m ({}/{})".format(l, i+1, len(synth_files)))

        second_lvl_stats[l] = {}

        df_synth = pd.read_csv(f, index_col=False)

        repl = False
        if len(df_synth) < len_df:
            repl = True

        df_synth = df_synth.sample(len_df, replace=repl).reset_index(drop=True)

        stats = stats_assessment(df_orig, df_synth, continuous_columns, aggregation_level, ignore_cols=cond_inputs)

        second_lvl_stats[l] = stats

    pickle.dump(second_lvl_stats, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for bias (1/76) already exists!
Results for ciDATGAN_1_1 (2/76) already exists!
Results for DATGAN_1_1 (3/76) already exists!
Results for normal_DATGAN_1_1 (4/76) already exists!
Results for ciDATGAN_1_2 (5/76) already exists!
Results for DATGAN_1_2 (6/76) already exists!
Results for normal_DATGAN_1_2 (7/76) already exists!
Results for ciDATGAN_1_3 (8/76) already exists!
Results for DATGAN_1_3 (9/76) already exists!
Results for normal_DATGAN_1_3 (10/76) already exists!
Results for ciDATGAN_1_4 (11/76) already exists!
Results for DATGAN_1_4 (12/76) already exists!
Results for normal_DATGAN_1_4 (13/76) already exists!
Results for ciDATGAN_1_5 (14/76) already exists!
Results for DATGAN_1_5 (15/76) already exists!
Results for normal_DATGAN_1_5 (16/76) already exists!
Results for ciDATGAN_2_1 (17/76) already exists!
Results for DATGAN_2_1 (18/76) already exists!
Results for normal_DATGAN_2_1 (19/76) already exists!
Results for ciDATGAN_2_2 (20/76) already exists!
Results for DATGAN_

In [156]:
res = {}

for s in stats_str:
    res[s] = {}

for m in second_lvl_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for c in second_lvl_stats[m].keys():
            res[s][m].append(second_lvl_stats[m][c][s])

In [157]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in second_lvl_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [159]:
sts = 'srmse'

res_DATGAN = []
res_ciDATGAN = []
res_normal = []

for i in range(5):
    for j in range(5):
        res_DATGAN.append(avg[sts]['DATGAN_{}_{}'.format(i+1, j+1)]['mean'])
        res_ciDATGAN.append(avg[sts]['ciDATGAN_{}_{}'.format(i+1, j+1)]['mean'])
        res_normal.append(avg[sts]['normal_DATGAN_{}_{}'.format(i+1, j+1)]['mean'])

tmp = [res_DATGAN, res_ciDATGAN, res_normal]
df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN', 'normal'])

plt.figure(figsize=(10,7))

sns.boxplot(data=df.T, palette=colors,showmeans=True,
            meanprops={"marker":"o",
                       "markerfacecolor":"white",
                       "markeredgecolor":"black",
                      "markersize":"10"})

val = avg[sts]['bias']['mean']

plt.plot([-0.3, 2.3], [val, val], 'k--')

plt.xticks([0,1,2], ['DATGAN', 'ciDATGAN', 'DATGAN - unbiased'])
plt.ylabel(sts.upper())

plt.savefig('../../figures/bias/lvl2.png'.format(s), bbox_inches='tight')
plt.savefig('../../figures/bias/lvl2.pdf'.format(s), bbox_inches='tight')
plt.close()

# ML efficacy

In [160]:
def check_low_appearing_vars(df):

    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()

def replace_low_appearing_values(df):

    dct_ = {}
    for i in df['hh_vehicles'].unique():
        if i >= 3:
            dct_[i] = '3+'
        else:
            dct_[i] = str(i)
    df['hh_vehicles'].replace(dct_, inplace=True)

    dct_ = {}
    for i in df['hh_people'].unique():
        if i >= 6:
            dct_[i] = '6+'
        else:
            dct_[i] = str(i)
    df['hh_people'].replace(dct_, inplace=True)

In [161]:
check_low_appearing_vars(df_orig)

Variable hh_vehicles: 
  4 - 0.52% (88)
  5 - 0.12% (21)
  6 - 0.02% (3)
  8 - 0.02% (3)
  7 - 0.01% (2)

Variable hh_people: 
  7 - 0.70% (118)
  8 - 0.18% (31)
  9 - 0.08% (13)
  10 - 0.03% (5)
  11 - 0.02% (3)



In [162]:
replace_low_appearing_values(df_orig)

In [163]:
check_low_appearing_vars(df_orig)

In [164]:
categorical_columns = list(set(df_orig.columns) - set(continuous_columns))

In [165]:
enc = OrdinalEncoder()
df_orig[categorical_columns] = enc.fit_transform(df_orig[categorical_columns])

In [166]:
pickle_name = 'ml_bias.pickle'

cv_modelscores = {}

try:
    cv_modelscores = pickle.load(open(results_path + pickle_name, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')

No previous results found, starting fresh


In [167]:
for i, (f, l) in enumerate(zip(synth_files, labels)):

    if l in cv_modelscores:
        print("Results for \033[1m{}\033[0m ({}/{}) already exists!".format(l, i+1, len(synth_files)))
    else:
        print("Preparing stats for \033[1m{}\033[0m ({}/{})".format(l, i+1, len(synth_files)))

        cv_modelscores[l] = {}

        # Load the synthetic dataset
        df_synth = pd.read_csv(f, index_col=False)
        repl = False
        if len(df_synth) < len_df:
            repl = True

        df_synth = df_synth.sample(len_df, replace=repl).reset_index(drop=True)

        # Replace the values rarely appearing
        replace_low_appearing_values(df_synth)

        # Encode the synthetic dataset
        df_synth[categorical_columns] = enc.transform(df_synth[categorical_columns])

        res = ml_assessment(df_orig, df_synth, continuous_columns, categorical_columns, ignore_cols=cond_inputs)

        cv_modelscores[l] = res

    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))

print("\033[1mFINISHED!\033[0m")

Preparing stats for bias (1/76)
Preparing stats for ciDATGAN_1_1 (2/76)
Preparing stats for DATGAN_1_1 (3/76)
Preparing stats for normal_DATGAN_1_1 (4/76)e (8/8)
      Preparing stats for ciDATGAN_1_2 (5/76)
Preparing stats for DATGAN_1_2 (6/76)
Preparing stats for normal_DATGAN_1_2 (7/76)
      Preparing stats for ciDATGAN_1_3 (8/76)
      Preparing stats for DATGAN_1_3 (9/76)
Preparing stats for normal_DATGAN_1_3 (10/76)
Preparing stats for ciDATGAN_1_4 (11/76)
Preparing stats for DATGAN_1_4 (12/76)
Preparing stats for normal_DATGAN_1_4 (13/76)
Preparing stats for ciDATGAN_1_5 (14/76)
Preparing stats for DATGAN_1_5 (15/76)
Preparing stats for normal_DATGAN_1_5 (16/76)
Preparing stats for ciDATGAN_2_1 (17/76)
Preparing stats for DATGAN_2_1 (18/76)
Preparing stats for normal_DATGAN_2_1 (19/76)
Preparing stats for ciDATGAN_2_2 (20/76)
Preparing stats for DATGAN_2_2 (21/76)
Preparing stats for normal_DATGAN_2_2 (22/76)
Preparing stats for ciDATGAN_2_3 (23/76)
Preparing stats for DATGAN_2

In [168]:
if 'original' in cv_modelscores:
    print("Results for \033[1m{}\033[0m already exists!".format('original'))
else:
    print("Preparing stats for \033[1m{}\033[0m".format('original'))

    res = ml_assessment(df_orig, df_orig, continuous_columns, categorical_columns)
    cv_modelscores['original'] = res
    pickle.dump(cv_modelscores, open(results_path + pickle_name, 'wb'))
    print("\033[1mFINISHED!\033[0m")

Preparing stats for original
               FINISHED!


In [169]:
res_ml = {
    'DATGAN': {'continuous': [], 'categorical': []},
    'ciDATGAN': {'continuous': [], 'categorical': []}
}

continuous_columns = list(set(continuous_columns) - set(cond_inputs))
categorical_columns = list(set(categorical_columns) - set(cond_inputs))

ori_scores = {col: cv_modelscores['original'][col]['test_log_loss'] for col in categorical_columns}
ori_scores.update({col: cv_modelscores['original'][col]['test_l2'] for col in continuous_columns})

res = {}

for c in ['continuous', 'categorical']:
    res[c] = {}

for m in cv_modelscores.keys():

    res['continuous'][m] = []
    for col in continuous_columns:
        res['continuous'][m].append(cv_modelscores[m][col]['original_l2']/ori_scores[col])

    res['categorical'][m] = []
    for col in categorical_columns:
        res['categorical'][m].append(cv_modelscores[m][col]['original_log_loss'] - ori_scores[col])

In [170]:
avg = {}

for c in ['continuous', 'categorical']:
    avg[c] = {}
    for f in res[c].keys():
        avg[c][f] = {
            'mean': np.mean(res[c][f]),
            'std': np.std(res[c][f])
        }

In [175]:
for c in ['continuous', 'categorical']:

    res_DATGAN = []
    res_ciDATGAN = []
    res_normal = []

    for i in range(5):
        for j in range(5):
            res_DATGAN.append(avg[c]['DATGAN_{}_{}'.format(i+1, j+1)]['mean'])
            res_ciDATGAN.append(avg[c]['ciDATGAN_{}_{}'.format(i+1, j+1)]['mean'])
            res_normal.append(avg[c]['normal_DATGAN_{}_{}'.format(i+1, j+1)]['mean'])


    res_DATGAN = np.array(res_DATGAN).flatten()
    res_ciDATGAN = np.array(res_ciDATGAN).flatten()
    res_normal = np.array(res_normal).flatten()

    # Remove values that are too big
    res_DATGAN = res_DATGAN[res_DATGAN < 120]
    res_ciDATGAN = res_ciDATGAN[res_ciDATGAN < 120]
    res_normal = res_normal[res_normal < 120]

    tmp = [res_DATGAN, res_ciDATGAN, res_normal]
    df = pd.DataFrame(tmp, index=['DATGAN', 'ciDATGAN', 'normal'])

    plt.figure(figsize=(10,7))

    sns.boxplot(data=df.T, palette=colors,showmeans=True,
                meanprops={"marker":"o",
                           "markerfacecolor":"white",
                           "markeredgecolor":"black",
                          "markersize":"10"})

    val = avg[c]['bias']['mean']

    plt.plot([-0.3, 2.3], [val, val], 'k--')

    plt.xticks([0,1,2], ['DATGAN', 'ciDATGAN', 'DATGAN - unbiased'])
    if c == 'continuous':
        plt.ylabel('Relative L2 error')
    else:
        plt.ylabel('Relative log loss')

    plt.savefig('../../figures/bias/ml_{}.png'.format(c[:3]), bbox_inches='tight')
    plt.savefig('../../figures/bias/ml_{}.pdf'.format(c[:3]), bbox_inches='tight')
    plt.close()